### Setup Reload

In [ ]:
%load_ext autoreload
%autoreload 2

### Import Dependency

In [ ]:
import torch
import numpy as np
import time
import h5py 

from os import path as osp
import os
import glob
from tqdm.notebook import tqdm

### Define path and device

In [ ]:
amass_dir = '/home/cotton_ahn/dataset/AMASS/'
h5py_dir = '/home/cotton_ahn/dataset/AMASS_h5py/'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using {}'.format(device))

### Make directory to save processed hdf5

In [ ]:
os.makedirs(h5py_dir, exist_ok=True)

### Setup the desired fps and str to bridge subject name and action name.

In [ ]:
desired_fps = 30
bridge_str = '+++'
gender_list = ['neutral', 'female', 'male']

In [ ]:
fps_set = set()
for fp in glob.glob(osp.join(amass_dir, '*')):
    dataset_name = fp.split('/')[-1]
    f = h5py.File(osp.join(h5py_dir, '{}.hdf5'.format(dataset_name)), 'a')
            
    print('===============Start Processing : {}==============='.format(dataset_name))
    for sub_fp in tqdm(sorted(glob.glob(osp.join(fp, '*/')))):
        for npz_fp in tqdm(sorted(glob.glob(osp.join(sub_fp, '*_stageii.npz')))):
            # read npz data
            bdata = np.load(npz_fp)
            gender = bdata['gender']
            trans = bdata['trans']
            betas = bdata['betas']
            poses = bdata['poses']
            frame_rate = bdata['mocap_frame_rate']
            
            # calculate stride for unified FPS
            stride = int(np.round(frame_rate / desired_fps))
            num_frames = len(range(0, trans.shape[0], stride))
            
            # add to .hdf5 group
            grp_name = npz_fp.split('/')[-2] + bridge_str + npz_fp.split('/')[-1].replace('_stageii.npz', '')
            grp = f.create_group(grp_name)
            grp.create_dataset('poses', data=poses[::stride])
            grp.create_dataset('trans', data=trans[::stride])
            grp.create_dataset('betas', data=betas)
            grp.create_dataset('gender', data=gender_list.index(gender))
